In [25]:
from pprint import pprint
from typing import Literal, List, Dict

from gen_ai_hub.proxy.native.openai import chat, completions
from pipeline.generate_tool_schema import generate_json_schema

In [26]:
################################
# Define Your Tool functions Here

def cosine(x: int) -> float:
    """
    Calculate the cosine of an integer 
    
    :param x: The input integer
    :return: A float representing the cosine of x
    """
    import math
    return math.cos(x)

def sin(x: int) -> float:
    """
    Calculate the sine of an integer

    :param x: The input integer
    :return: A float representing the sine of x
    """
    import math
    return math.sin(x)

def addition(x: float, y: float) -> float:
    """
    Calculate the addition of two float numbers

    :param x: The input float x
    :param y: The input float y
    :return: A float representing the sum of x and y
    """
    return x + y

In [27]:
def run_loop(prompt, tools):
    import json
    continue_loop = True
    
    messages = []
    system_message = {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."}
    user_message =  {"role": "user", "content": prompt}
    messages.append(system_message)
    messages.append(user_message)

    while continue_loop: 
        print(messages) # Comment out this line for debug purpose
        response = chat.completions.create( 
            messages = messages,
            model_name = "gpt-4",
            tools=tools)
        response_message = response.choices[0].message
        
        if response.choices[0].finish_reason == "stop":
            return response_message.content

        tool_call_block = response_message.tool_calls[0]
        if tool_call_block:
            messages.append({"role": "assistant", "tool_calls": response_message.tool_calls})
            tool_call_id = tool_call_block.id
            tool_call_function_name = tool_call_block.function.name
            tool_call_function_arguments = json.loads(tool_call_block.function.arguments)
            
            if tool_call_function_name == 'cosine':
                x = tool_call_function_arguments.get('x')
                tool_call_result = cosine(x),
                function_call_result_message = {
                    "role" : "tool", 
                    "content": json.dumps({
                        "x": x, 
                        "result": tool_call_result
                    }), 
                "tool_call_id": tool_call_id
                }
                messages.append(function_call_result_message) 

            elif tool_call_function_name == 'sin':
                x = tool_call_function_arguments.get('x')
                tool_call_result = sin(x),
                function_call_result_message = {
                    "role" : "tool", 
                    "content": json.dumps({
                        "x": x, 
                        "result": tool_call_result
                    }), 
                "tool_call_id": tool_call_id
                }
                messages.append(function_call_result_message) 

            elif tool_call_function_name == 'addition':
                x = tool_call_function_arguments.get('x')
                y = tool_call_function_arguments.get('y')
                tool_call_result = addition(x, y),
                function_call_result_message = {
                    "role" : "tool", 
                    "content": json.dumps({
                        "x": x,
                        "y": y, 
                        "result": tool_call_result
                    }), 
                "tool_call_id": tool_call_id
                }
                messages.append(function_call_result_message) 


In [28]:
tools = []
tools.append(generate_json_schema(cosine))
tools.append(generate_json_schema(sin))
tools.append(generate_json_schema(addition))
run_loop(prompt = "Calculate cos(3) + sin(4) for me", tools = tools)

[{'role': 'system', 'content': 'You are a helpful customer support assistant. Use the supplied tools to assist the user.'}, {'role': 'user', 'content': 'Calculate cos(3) + sin(4) for me'}]
[{'role': 'system', 'content': 'You are a helpful customer support assistant. Use the supplied tools to assist the user.'}, {'role': 'user', 'content': 'Calculate cos(3) + sin(4) for me'}, {'role': 'assistant', 'tool_calls': [ChatCompletionMessageToolCall(id='call_cisWyewYiIYWh1L6NncUIeiL', function=Function(arguments='{"x":3}', name='cosine'), type='function')]}, {'role': 'tool', 'content': '{"x": 3, "result": [-0.9899924966004454]}', 'tool_call_id': 'call_cisWyewYiIYWh1L6NncUIeiL'}]
[{'role': 'system', 'content': 'You are a helpful customer support assistant. Use the supplied tools to assist the user.'}, {'role': 'user', 'content': 'Calculate cos(3) + sin(4) for me'}, {'role': 'assistant', 'tool_calls': [ChatCompletionMessageToolCall(id='call_cisWyewYiIYWh1L6NncUIeiL', function=Function(arguments='

'The value of \\( \\cos(3) + \\sin(4) \\) is approximately -1.74679.'